In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [11]:
data = pd.read_csv('./time_series_covid19_confirmed_global.csv')

In [12]:
data.drop('Province/State', axis=1, inplace=True)
data.drop('Lat', axis=1, inplace=True)
data.drop('Long', axis=1, inplace=True)
data.columns = np.concatenate((np.array(['Country/Region']),np.arange(len(data.columns[1:]))))

In [13]:
data

,Country/Region,0,1,2,3,4,5,6,7,8,...,608,609,610,611,612,613,614,615,616,617
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,154712,154757,154800,154960,154960,154960,155072,155093,155128,155174
1,Albania,0,0,0,0,0,0,0,0,0,...,164276,165096,165864,166690,167354,167893,168188,168782,169462,170131
2,Algeria,0,0,0,0,0,0,0,0,0,...,201948,202122,202283,202449,202574,202722,202877,203045,203198,203359
3,Andorra,0,0,0,0,0,0,0,0,0,...,15140,15153,15156,15167,15167,15167,15189,15192,15209,15222
4,Angola,0,0,0,0,0,0,0,0,0,...,52968,53387,53840,54280,54795,55121,55583,56040,56583,56583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,Vietnam,0,2,2,2,2,2,2,2,2,...,707436,718963,728435,736972,746678,756689,766051,770640,779398,790755
275,West Bank and Gaza,0,0,0,0,0,0,0,0,0,...,388470,390369,392452,394683,395677,396746,398946,400649,402255,403716
276,Yemen,0,0,0,0,0,0,0,0,0,...,8752,8789,8830,8861,8891,8934,8988,9016,9039,9067
277,Zambia,0,0,0,0,0,0,0,0,0,...,208599,208676,208715,208778,208829,208857,208867,208912,209002,209046


In [14]:
Confirmed = {'Brazil':[],'India':[],'United Kingdom':[],'Russia':[],'US':[]}
time_series = {'Brazil':[],'India':[],'United Kingdom':[],'Russia':[],'US':[]}
for index,row in data.iterrows():
    for key in Confirmed:
        if row['Country/Region'] == key:
            d = data.iloc[index][1:].astype(np.float32).values
            time_series[key].append(d)
            m = np.zeros_like(d)
            for j in range(1,len(d)):
                m[j] = d[j] - d[j-1]
            Confirmed[key].append(m)

In [19]:
time_series['United Kingdom'] = [np.sum(np.array(time_series['United Kingdom']),axis=0)]

In [28]:
np.array(time_series['United Kingdom']).shape

(1, 618)

In [ ]:
plt.figure()
plt.plot(np.array(time_series['Brazil']).squeeze())
plt.plot(np.array(time_series['India']).squeeze())
plt.plot(np.array(time_series['United Kingdom']).squeeze())
plt.plot(np.array(time_series['Russia']).squeeze())
plt.plot(np.array(time_series['US']).squeeze())
plt.legend(['Brazil','India','UK','Russia','US'])
plt.xlabel('day')
plt.ylabel('Number of confirmed people')

In [6]:
print(len(Confirmed['Brazil']))
print(len(Confirmed['India']))
print(len(Confirmed['United Kingdom']))
print(len(Confirmed['US']))
print(len(Confirmed['Russia']))
timeLength = len(Confirmed['Brazil'][0])
print(timeLength)

1
1
12
1
1
618


In [7]:
Confirmed['United Kingdom'] = [np.sum(np.array(Confirmed['United Kingdom']),axis=0)]

In [8]:
plt.plot(Confirmed['Russia'][0])
plt.title('Number of confirmed cases in a single day in Russia')
plt.xlabel('day')

Text(0.5, 0, 'day')

In [9]:
sequence_length= 10
delay = 1
def func(country):
    dataCountry = []
    for i in range(timeLength - sequence_length -delay + 1):
        dataCountry.append(Confirmed[country][0][i:i+sequence_length+delay])
    dataCountry = np.array(dataCountry)
    x_train = dataCountry[:,:sequence_length]
    y_train = dataCountry[:,-1]
    return np.expand_dims(x_train,2),y_train

In [10]:
x_Brazil,y_Brazil = func('Brazil')
x_India,y_India = func('India')
x_Russia,y_Russia = func('Russia')
x_UK,y_UK = func('United Kingdom')
x_US,y_US = func('US')

In [11]:
def norm(country,x,y):
    x = x / np.max(Confirmed[country][0])
    y = y / np.max(Confirmed[country][0])
    return x, y

In [12]:
x_Brazil, y_Brazil = norm('Brazil',x_Brazil, y_Brazil)
x_India, y_India = norm('India',x_India, y_India)
x_Russia, y_Russia = norm('Russia',x_Russia, y_Russia)
x_UK, y_UK = norm('United Kingdom',x_UK, y_UK)
x_US, y_US = norm('US',x_US,y_US)

In [13]:
from sklearn.model_selection import train_test_split
train_x_Brazil,test_x_Brazil,train_y_Brazil,test_y_Brazil = train_test_split(x_Brazil,y_Brazil)
train_x_India,test_x_India,train_y_India,test_y_India = train_test_split(x_India,y_India)
train_x_Russia,test_x_Russia,train_y_Russia,test_y_Russia = train_test_split(x_Russia,y_Russia)
train_x_UK,test_x_UK,train_y_UK,test_y_UK = train_test_split(x_UK,y_UK)
train_x_US,test_x_US,train_y_US,test_y_US = train_test_split(x_US,y_US)

In [14]:
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

    def __len__(self):
        return len(self.features)

In [15]:
train_Brazil_dataset = Mydataset(train_x_Brazil,train_y_Brazil)
test_Brazil_dataset = Mydataset(test_x_Brazil,test_y_Brazil)
train_India_dataset = Mydataset(train_x_India,train_y_India)
test_India_dataset = Mydataset(test_x_India,test_y_India)
train_Russia_dataset = Mydataset(train_x_Russia,train_y_Russia)
test_Russia_dataset = Mydataset(test_x_Russia,test_y_Russia)
train_UK_dataset = Mydataset(train_x_UK,train_y_UK)
test_UK_dataset = Mydataset(test_x_UK,test_y_UK)
train_US_dataset = Mydataset(train_x_US,train_y_US)
test_US_dataset = Mydataset(test_x_US,test_y_US)

In [16]:
BTACH_SIZE = 64
train_Brazil_dl = torch.utils.data.DataLoader(train_Brazil_dataset,batch_size=BTACH_SIZE,shuffle=True)
test_Brazil_dl = torch.utils.data.DataLoader(test_Brazil_dataset,batch_size=BTACH_SIZE)
train_India_dl = torch.utils.data.DataLoader(train_India_dataset,batch_size=BTACH_SIZE,shuffle=True)
test_India_dl = torch.utils.data.DataLoader(test_India_dataset,batch_size=BTACH_SIZE)
train_Russia_dl = torch.utils.data.DataLoader(train_Russia_dataset,batch_size=BTACH_SIZE,shuffle=True)
test_Russia_dl = torch.utils.data.DataLoader(test_Russia_dataset,batch_size=BTACH_SIZE)
train_UK_dl = torch.utils.data.DataLoader(train_UK_dataset,batch_size=BTACH_SIZE,shuffle=True)
test_UK_dl = torch.utils.data.DataLoader(test_UK_dataset,batch_size=BTACH_SIZE)
train_US_dl = torch.utils.data.DataLoader(train_US_dataset,batch_size=BTACH_SIZE,shuffle=True)
test_US_dl = torch.utils.data.DataLoader(test_US_dataset,batch_size=BTACH_SIZE)

In [17]:
hidden_size = 128
class Net(nn.Module):
    # GRU实现 
    def __init__(self,hidden_size):
        super().__init__()
        self.num_layers = 3
        self.rnn = nn.GRU(1,hidden_size,num_layers=self.num_layers,batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 1)
        self.coff = nn.Parameter(torch.ones(1).squeeze()/2,requires_grad=True)
    def forward(self,inputs):
        _,hn = self.rnn(inputs)
        x = F.dropout(F.relu(self.fc1(hn)))
        x = self.fc2(x)
        x = torch.sum(x,axis=0)
        x = F.relu(x)

        with torch.no_grad():
            m = list() 
            for ele in inputs:
                device = 'cuda' if torch.cuda.is_available() else 'cpu'
                if device == 'cuda':
                    ele = ele.squeeze().cpu().detach().numpy()
                else:
                    ele = ele.squeeze().detach().numpy()
                res = self.ES_Forecast(ele,1)
                m.append([res])
            m = torch.as_tensor(m,dtype=torch.float32).to(device)
        x = torch.cat((x,m),dim=1)
        x = self.coff*x[:,0]+(1 - self.coff)*x[:,1]
#         x = torch.mean(x,dim=1)
#         x = self.mix1(x)
#         x = self.mix2(x)
        x = F.relu(x)
        return x.squeeze()
    def ES_Forecast(self,A,T):
        yt,n = A,len(A)
        alpha, st1_0 = 0.3,np.mean(yt[0:3])
        st2_0, st3_0 = st1_0, st1_0
        st1,st2,st3 = list(),list(),list()
        st1.append(alpha*yt[0]+(1-alpha)*st1_0)
        st2.append(alpha*st1[0]+(1-alpha)*st2_0)
        st3.append(alpha*st2[0]+(1-alpha)*st3_0)
        for i in range(1,n):
            st1.append(alpha*yt[i]+(1-alpha)*st1[i-1])
            st2.append(alpha*st1[i]+(1-alpha)*st2[i-1])
            st3.append(alpha*st2[i]+(1-alpha)*st3[i-1])
        a = 3*st1[-1] - 3*st2[-1]+st3[-1]
        b = 0.5*alpha/((1-alpha)**2)*((6-5*alpha)*st1[-1]-2*(5-4*alpha)*st2[-1]+(4-3*alpha)*st3[-1])
        c = 0.5*alpha**2/(1-alpha)**2*(st1[-1]-2*st2[-1]+st3[-1])
        coff = [c,b,a]
        res = c*T**2+b*T+a
        return res
        
# class Net(nn.Module):
#     # LSTM实现 
#     def __init__(self, hidden_size):
#         super(Net, self).__init__()
#         self.rnn = nn.LSTM(1, hidden_size,num_layers=4,batch_first=True)
#         self.fc1 = nn.Linear(hidden_size, 256)
#         self.fc2 = nn.Linear(256, 1)

#     def forward(self, inputs):
#         _, s_o = self.rnn(inputs)
#         s_o = s_o[-1]
#         x = F.dropout(F.relu(self.fc1(s_o)))
#         x = self.fc2(x)
#         x = torch.sum(x,axis=0)
#         return torch.squeeze(x)

In [31]:
model_Brazil = Net(hidden_size)
model_India = Net(hidden_size)
model_Russia = Net(hidden_size)
model_UK = Net(hidden_size)
model_US = Net(hidden_size)

In [32]:
if torch.cuda.is_available():
    model_India.to('cuda')
    model_Brazil.to('cuda')
    model_Russia.to('cuda')
    model_UK.to('cuda')
    model_US.to('cuda')

In [33]:
loss_fn = nn.MSELoss()
optimizer_Brazil = torch.optim.Adam(model_Brazil.parameters(), lr=5e-4)
optimizer_India = torch.optim.Adam(model_India.parameters(), lr=5e-4)
optimizer_Russia = torch.optim.Adam(model_Russia.parameters(), lr=5e-4)
optimizer_UK = torch.optim.Adam(model_UK.parameters(), lr=5e-4)
optimizer_US = torch.optim.Adam(model_US.parameters(), lr=5e-4)

In [21]:
def fit(epoch, model, trainloader, testloader,optimizer):
    total = 0
    running_loss = 0
    
    model.train()
    for x, y in trainloader:
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            total += y.size(0)
            running_loss += loss.item()
#    exp_lr_scheduler.step()
    epoch_loss = running_loss / len(trainloader.dataset)
        
        
    test_total = 0
    test_running_loss = 0 
    
    model.eval()
    with torch.no_grad():
        for x, y in testloader:
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            test_total += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset)
        
#     print('epoch: ', epoch, 
#           'loss： ', round(epoch_loss, 6),
#           'test_loss： ', round(epoch_test_loss, 6),
#              )
        
    return epoch_loss, epoch_test_loss

In [22]:
def pltLoss(train_loss,test_loss,name):
    plt.figure()
    plt.plot(train_loss)
    plt.plot(test_loss)
    plt.legend(['train_loss','test_loss'])
    plt.xlabel('epochs')
    plt.title(name)

In [23]:
train_loss = []
test_loss = []
epochs = 200
bestModel_Brazil = None
bestLoss = np.inf
for epoch in tqdm(range(epochs),total=epochs,smoothing=0.9):
    epoch_loss, epoch_test_loss = fit(epoch,model_Brazil,train_Brazil_dl,test_Brazil_dl,optimizer_Brazil)
    train_loss.append(epoch_loss)
    test_loss.append(epoch_test_loss)
    if epoch_test_loss<bestLoss:
        bestLoss = epoch_test_loss
        bestModel_Brazil = model_Brazil

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:51<00:00,  3.91it/s]


In [70]:
pltLoss(train_loss,test_loss,'Brazil')

In [24]:
train_loss = []
test_loss = []
epochs = 200
bestModel_India = None
bestLoss = np.inf
for epoch in tqdm(range(epochs),total=epochs,smoothing=0.9):
    epoch_loss, epoch_test_loss = fit(epoch,model_India,train_India_dl,test_India_dl,optimizer_India)
    train_loss.append(epoch_loss)
    test_loss.append(epoch_test_loss)
    if epoch_test_loss < bestLoss:
        bestLoss = epoch_test_loss
        bestModel_India = model_India

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:50<00:00,  3.96it/s]


In [72]:
pltLoss(train_loss,test_loss,'India')

In [25]:
train_loss = []
test_loss = []
epochs = 200
bestModel_Russia = None
bestLoss = np.inf
for epoch in tqdm(range(epochs),total=epochs,smoothing=0.9):
    epoch_loss, epoch_test_loss = fit(epoch,model_Russia,train_Russia_dl,test_Russia_dl,optimizer_Russia)
    train_loss.append(epoch_loss)
    test_loss.append(epoch_test_loss)
    if epoch_test_loss<bestLoss:
        bestLoss = epoch_test_loss
        bestModel_Russia = model_Russia

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:50<00:00,  3.96it/s]


In [ ]:
pltLoss(train_loss,test_loss,'Russia')

In [26]:
train_loss = []
test_loss = []
epochs = 200
bestModel_UK = None
bestLoss = np.inf
for epoch in tqdm(range(epochs),total=epochs,smoothing=0.9):
    epoch_loss, epoch_test_loss = fit(epoch,model_UK,train_UK_dl,test_UK_dl,optimizer_UK)
    train_loss.append(epoch_loss)
    test_loss.append(epoch_test_loss)
    if epoch_test_loss<bestLoss:
        bestLoss = epoch_test_loss
        bestModel_UK = model_UK

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:50<00:00,  3.96it/s]


In [ ]:
pltLoss(train_loss,test_loss,'UK')

In [34]:
train_loss = []
test_loss = []
epochs = 200
bestModel_US = None
bestLoss = np.inf
for epoch in tqdm(range(epochs),total=epochs,smoothing=0.9):
    epoch_loss, epoch_test_loss = fit(epoch,model_US,train_US_dl,test_US_dl,optimizer_US)
    train_loss.append(epoch_loss)
    test_loss.append(epoch_test_loss)
    if epoch_test_loss<bestLoss:
        bestLoss = epoch_test_loss
        bestModel_US = model_US

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:50<00:00,  3.94it/s]


In [ ]:
pltLoss(train_loss,test_loss,'US')

In [68]:
n = train_x_Brazil.shape[0]

In [77]:
plt.figure()
plt.scatter(range(1,n+1),bestModel_US(torch.from_numpy(train_x_US).to('cuda')).detach().cpu().numpy())
plt.scatter(range(1,n+1),train_y_US)
plt.legend(['Predicted','Ground truth'])
plt.title('US')
plt.xlabel('day')

Text(0.5, 0, 'day')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


# India Brazil Russia UK US未来预测

In [35]:
x4pred_bra = Confirmed['Brazil'][0][-sequence_length:]/np.max(Confirmed['Brazil'][0])
x4pred_ind = Confirmed['India'][0][-sequence_length:]/np.max(Confirmed['India'][0])
x4pred_rus = Confirmed['Russia'][0][-sequence_length:]/np.max(Confirmed['Russia'][0])
x4pred_UK = Confirmed['United Kingdom'][0][-sequence_length:]/np.max(Confirmed['United Kingdom'][0])
x4pred_US = Confirmed['US'][0][-sequence_length:]/np.max(Confirmed['US'][0])
ptr = 0
POINT_NUM = 61
while ptr<POINT_NUM:
    x_input = np.expand_dims(x4pred_bra[ptr:ptr+sequence_length],(0,2))
    out = bestModel_Brazil(torch.as_tensor(x_input,dtype=torch.float).cuda())
    out = out.cpu().detach().numpy().reshape(1,)
    x4pred_bra = np.concatenate((x4pred_bra,out))
    
    x_input = np.expand_dims(x4pred_ind[ptr:ptr+sequence_length],(0,2))
    out = bestModel_India(torch.as_tensor(x_input,dtype=torch.float).cuda())
    out = out.cpu().detach().numpy().reshape(1,)
    x4pred_ind = np.concatenate((x4pred_ind,out))
    
    x_input = np.expand_dims(x4pred_rus[ptr:ptr+sequence_length],(0,2))
    out = bestModel_Russia(torch.as_tensor(x_input,dtype=torch.float).cuda())
    out = out.cpu().detach().numpy().reshape(1,)
    x4pred_rus = np.concatenate((x4pred_rus,out))
    
    x_input = np.expand_dims(x4pred_US[ptr:ptr+sequence_length],(0,2))
    out = bestModel_US(torch.as_tensor(x_input,dtype=torch.float).cuda())
    out = out.cpu().detach().numpy().reshape(1,)
    x4pred_US = np.concatenate((x4pred_US,out))
    
    x_input = np.expand_dims(x4pred_UK[ptr:ptr+sequence_length],(0,2))
    out = bestModel_UK(torch.as_tensor(x_input,dtype=torch.float).cuda())
    out = out.cpu().detach().numpy().reshape(1,)
    x4pred_UK = np.concatenate((x4pred_UK,out))
    
    ptr += 1
x4pred_bra = x4pred_bra[-POINT_NUM:]*np.max(Confirmed['Brazil'][0])
x4pred_ind = x4pred_ind[-POINT_NUM:]*np.max(Confirmed['India'][0])
x4pred_rus = x4pred_rus[-POINT_NUM:]*np.max(Confirmed['Russia'][0])
x4pred_US = x4pred_US[-POINT_NUM:]*np.max(Confirmed['US'][0])
x4pred_UK = x4pred_UK[-POINT_NUM:]*np.max(Confirmed['United Kingdom'][0])

In [36]:
def mPlot(string,d):
    plt.figure()
    plt.plot(np.arange(timeLength),Confirmed[string][0])
    plt.plot(np.arange(timeLength,timeLength+POINT_NUM),d)
    plt.legend(['Known','Predicted'])
    plt.xlabel('day')
    plt.title(string)

In [37]:
mPlot('Brazil',x4pred_bra)
mPlot('India',x4pred_ind)
mPlot('Russia',x4pred_rus)
mPlot('United Kingdom',x4pred_UK)
mPlot('US',x4pred_US)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [38]:
def calRes(x1,x2):
    temp = np.concatenate((x1,x2))
    return np.cumsum(temp)

In [39]:
bra = calRes(Confirmed['Brazil'][0],x4pred_bra)
ind = calRes(Confirmed['India'][0],x4pred_ind)
rus = calRes(Confirmed['Russia'][0],x4pred_rus)
UK = calRes(Confirmed['United Kingdom'][0],x4pred_UK)
US = calRes(Confirmed['US'][0],x4pred_US)

In [43]:
plt.figure()
plt.plot(bra)
plt.plot(ind)
plt.plot(rus)
plt.plot(UK)
plt.plot(US)
plt.scatter([timeLength for _ in range(50)],np.linspace(0,np.max(US)),s=1)
plt.legend(['Brazil','India','Russia','UK','US'])
plt.xlabel('day')
plt.ylabel('Total confirmed')

Text(0, 0.5, 'Total confirmed')

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


# 生成结果csv

In [47]:
x4pred_bra = bra[-31:]
x4pred_ind = ind[-31:]
x4pred_rus = rus[-31:]
x4pred_UK = UK[-31:]
x4pred_US = US[-31:]

In [48]:
df=pd.DataFrame(columns=["Brazil","India","Russia","United Kindom","United States"])

In [49]:
df["Brazil"] = x4pred_bra
df["India"] = x4pred_ind
df["Russia"] = x4pred_rus
df["United Kindom"] = x4pred_UK
df["United States"] = x4pred_US

In [52]:
df.to_csv('result.csv',index=None)